# Logging unused variables

Custom logger wrappers in *Sinergym* enable recording any metric based on interaction data with the environment (as shown [here](https://ugr-sail.github.io/sinergym/compilation/main/pages/notebooks/personalize_loggerwrapper.html)). However, there may be cases where you want to monitor data directly from the simulator.

Until version v3.3.2 of *Sinergym*, all variables monitored using the `LoggerWrapper` had to be present in the observation space. This represents a drawback when we want to monitor certain variables of the simulation that are not used for control (i.e., that are not present in the environment's observation space).

Including extra variables that are not directly part of the observation space requires certain internal changes that break the minimalist structure of the classes and EnergyPlus API usage that make up the tool.

This notebook explains the correct way to do it, which is possible since *Sinergym* v3.3.3. It involves the use of the [ReduceObservationWrapper](https://ugr-sail.github.io/sinergym/compilation/main/pages/wrappers.html#reduceobservationwrapper) in combination with the [LoggerWrapper](https://ugr-sail.github.io/sinergym/compilation/main/pages/wrappers.html#loggerwrapper).

The idea is to define all the variables to be observed, whether they are part of the final observation space or not, and monitor everything with the `LoggerWrapper`, to later add a layer that removes the desired variables from the observation space (just when they are already being monitored, which is our goal).

In [ ]:
import gymnasium as gym
import numpy as np

import sinergym
from sinergym.utils.wrappers import (
    LoggerWrapper,
    NormalizeAction,
    NormalizeObservation,
    ReduceObservationWrapper,
    CSVLogger,
    WandBLogger)

# Creating environment and applying wrappers for normalization and logging
env = gym.make('Eplus-5zone-hot-continuous-stochastic-v1')
env = NormalizeAction(env)
env = NormalizeObservation(env)
env = LoggerWrapper(env)
print('###########################################################################')
print('Old observation space shape: ', env.observation_space.shape[0])
print('Old observation variables: ',
      env.get_wrapper_attr('observation_variables'))
print('###########################################################################')

# Here we can add the CSVLogger or WandBLogger
env = CSVLogger(env)
env = WandBLogger(env,
                  entity='sail_ugr',
                  project_name='test-porject',
                  run_name='test-reduction')

env = ReduceObservationWrapper(env, obs_reduction=[
                               'outdoor_temperature', 'outdoor_humidity', 'air_temperature'])
print('###########################################################################')
print('Wrapped observation space shape: ', env.observation_space.shape[0])
print('Wrapped observation variables: ',
      env.get_wrapper_attr('observation_variables'))
print('###########################################################################')

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.
[ENVIRONMENT] (INFO) : Name: 5zone-hot-continuous-stochastic-v1
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created.
[MODELING] (INFO) : Working directory: /workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-stochastic-v1-res1
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Update building model Output:Variable with variable names.
[MODELING] (INFO) : Update building model Output:Meter with meter names.
[MODELING] (INFO) : Runperiod established.
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[REWARD] (INFO) : Reward function initialized.
[ENVIRONMENT] (INFO) : Environment created successfully.
[WRAPPER 

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: alex_ugr (sail_ugr). Use `wandb login --relogin` to force relogin


[WRAPPER WandBLogger] (INFO) : Wrapper initialized.
[WRAPPER ReduceObservationWrapper] (INFO) : Wrapper initialized.
###########################################################################
Wrapped observation space shape:  14
Wrapped observation variables:  ['month', 'day_of_month', 'hour', 'wind_speed', 'wind_direction', 'diffuse_solar_radiation', 'direct_solar_radiation', 'htg_setpoint', 'clg_setpoint', 'air_humidity', 'people_occupant', 'co2_emission', 'HVAC_electricity_demand_rate', 'total_electricity_HVAC']
###########################################################################


The order of the wrappers is important! By applying normalization first, for example, we ensure that this transformation is subsequently monitored. 

As we apply the logger before reducing the observation space, we also record these variables before they are removed from the observations. If we use the logger wrapper at the end, these variables would not be monitored. This can be verified by reviewing the CSV files generated for the observation variables.

Let's review the info dictionary to see the normalized variables that are no longer in `obs`:

In [ ]:
obs, info = env.reset()
print('###########################################################################')
print('Reset observation length: ', len(obs))
print('Removed variables info: ', info['removed_observation'])
print('###########################################################################')

a = env.action_space.sample()
obs, _, _, _, info = env.step(a)
print('###########################################################################')
print('step observation length: ', len(obs))
print('Removed variables info: ', info['removed_observation'])
print('###########################################################################')

terminated = truncated = False
while not (terminated or truncated):
    a = env.action_space.sample()
    _, _, terminated, truncated, _ = env.step(a)

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode.
[ENVIRONMENT] (INFO) : Episode 1: 5zone-hot-continuous-stochastic-v1
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created.
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Adapting weather to building model.
[MODELING] (INFO) : Weather noise applied in columns: ['drybulb']
[ENVIRONMENT] (INFO) : Saving episode output path.
[ENVIRONMENT] (INFO) : Episode 1 started.
[SIMULATOR] (INFO) : handlers initialized.
[SIMULATOR] (INFO) : handlers are ready.
[SIMULATOR] (INFO) : System is ready.
[WRAPPER NormalizeObservation] (INFO) : Saving normalization calibration data.
###########################################################################
Reset observation length:  14
Removed variables info:  {'outdoor_temper

Note that even if we remove a variable that is used in the reward function, as this value is used in the core of the environment (before any wrapper), it still works as expected.

In [3]:
env.close()

[WRAPPER WandBLogger] (INFO) : Environment closed, dumping summary metrics in WandB Platform.
Simulation Progress [Episode 1]: 100%|██████████| 100/100 [00:16<00:00,  6.90%/s, 100% completed]

wandb: Adding directory to artifact (/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-stochastic-v1-res1)... Done. 0.0s


Agent_actions/Cooling_Setpoint_RL,▃▆▇▇▄▁█▄▄▃▂▇▅█▇▃▄█▇▄▅▅█▅█▄█▂▃▂█▂▄▃▃
Agent_actions/Heating_Setpoint_RL,▆█▁█▂▁▃▃█▆█▂▅▆▃▂▇▆▃▅▅▅▇▄▃▆▇▃▄▇▃▇▆▃▃
Info/abs_comfort_penalty,▃▇█▃█▇██▃█▅█▁█▆█▇██▇▇█▃█▄██▇████▄█▆
Info/abs_energy_penalty,██▁██▆██▇█▃█▆▇█▂█▇▇▆▇█▇█▇██▆██████▅
Info/comfort_term,▃▇█▃█▇██▃█▅█▁█▆█▇██▇▇█▃█▄██▇████▄█▆
Info/energy_term,██▁██▆██▇█▃█▆▇█▂█▇▇▆▇█▇█▇██▆██████▅
Info/total_power_demand,▁▁█▁▁▃▁▁▂▁▆▁▃▂▁▇▁▂▂▃▂▁▂▁▂▁▁▃▁▁▁▁▁▁▄
Info/total_temperature_violation,▆▂▁▆▁▂▁▁▆▁▄▁█▁▃▁▂▁▁▂▂▁▆▁▅▁▁▂▁▁▁▁▅▁▃
Normalized_observations/HVAC_electricity_demand_rate,▁▁█▁▁▃▁▂▂▂▇▂▃▂▁█▂▂▂▃▂▁▂▁▂▁▁▃▁▁▁▁▁▁▅
Normalized_observations/air_humidity,▆▆▄█▇▅▆▃▄▁▅▁▃▁▃▂▅▂▆▂▇▂▆▂▄▃▅▃▅▃▅▃▃▃▄
Normalized_observations/air_temperature,▆▄▂▇▂▅▃▃▆▃▅▄▇▄▂▄▃▄▃▃▆▃█▃█▃▁▄▂▃▃▁▅▁▅


[WRAPPER CSVLogger] (INFO) : Environment closed, data updated in monitor and progress.csv.
Simulation Progress [Episode 1]: 100%|██████████| 100/100 [00:29<00:00,  3.37%/s, 100% completed]
[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-continuous-stochastic-v1]
[WRAPPER NormalizeObservation] (INFO) : Saving normalization calibration data.


/usr/local/lib/python3.12/dist-packages/wandb/sdk/wandb_run.py:2372: UserWarning: Run (rl1s4e17) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stdout", data),
